In [3]:
from imports import*

from mf_sim import *
from mf_fit import *
from mf_predict import *

from mb_sim import *
from mb_fit import *
from mb_predict import *

from hybrid_sim import *
from hybrid_fit import *
from hybrid_predict import *

from nws_sim import *
from nws_fit import *
from nws_predict import *

from pwsls_sim import *
from pwsls_fit import *
from pwsls_predict import *

from kdh_sim import *
from kdh_fit import *
from kdh_predict import *

from utils import *
from plot_stats import *
from logistic_regression import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from tensorflow.python.keras.losses import binary_crossentropy

Init Plugin
Init Graph Optimizer
Init Kernel


In [4]:
#config num of agent to simulate
num_of_agents = 1

#config num of trails for each block
num_of_trials = 100

num_of_block = 2
# for cross valdation 
array = np.arange(num_of_block)
cv = [np.roll(array,i) for i in range(num_of_block)]
cv = np.array(cv)

reversal = 50

# create transtion probs
transtion_probs = np.array([[.8,.2],
                            [.2,.8]])

In [8]:
# mf
parameters = []
data_per_agent = []

data_results = {
               'agent': [], 
               'fit_parameters_mf': [], 
               'train_nlp_mf' : [], 
               'test_acc_mf': [],
               'test_nlp_mf': [], 
    
               'fit_parameters_mb': [], 
               'train_nlp_mb' : [], 
               'test_acc_mb': [],
               'test_nlp_mb': [],
    
               'fit_parameters_hybrid': [], 
               'train_nlp_hybrid' : [], 
               'test_acc_hybrid': [],
               'test_nlp_hybrid': [], 
    
               'fit_parameters_nws': [], 
               'train_nlp_nws' : [], 
               'test_acc_nws': [],
               'test_nlp_nws': [], 
    
               'fit_parameters_pwsls': [], 
               'train_nlp_pwsls' : [], 
               'test_acc_pwsls': [],
               'test_nlp_pwsls': [], 
    
               'fit_parameters_kdh': [], 
               'train_nlp_kdh' : [], 
               'test_acc_kdh': [],
               'test_nlp_kdh': [], 
    
               'fit_parameters_logistic_regression': [], 
               'train_nlp_logistic_regression' : [], 
               'test_acc_logistic_regression': [],
               'test_nlp_logistic_regression': [],
 
        }
for agent in tqdm(range(num_of_agents)):
    data = []
    param = configuration_parameters_mf()
    parameters.append(param)

    for i in range(num_of_block):
        # create rewards probs 
        reward_probs = create_reward_probs(num_of_trials,reversal,0.2,0.8)
        df = mf_sim(
                        param,
                        num_of_trials,
                        transtion_probs,
                        reward_probs
        ) 
        data.append(df)
    data_per_agent.append(data)
    print(f'*** Fit ***')
    for n,t in enumerate(cv):
        data_results['agent'].append(agent)
        print(f'* loocv block {n} *')
        train_arr = t[0:-1]
        test_arr = t[-1:]
       
        # split train and test data
        train_data = [data_per_agent[agent][sim] for sim in train_arr]
        train_data = pd.concat(train_data) 
        train_data.reset_index(inplace=True)
        n_train = len(train_data)

        test_data = [data_per_agent[agent][sim] for sim in test_arr]
        test_data = pd.concat(test_data) 
        test_data.reset_index(inplace=True)
        n_test = len(test_data)
        
        print('* fit mf *')
        res = mf_fit(train_data,2)
        data_results['fit_parameters_mf'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mf'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mf_predict(test_data,res.x,)
        
        # Test Accuracy
        data_results['test_acc_mf'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mf'].append((running_loss/n_test).numpy())
        
        print('* fit mb *')
        res = mb_fit(train_data,2)
        data_results['fit_parameters_mb'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mb'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mb_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_mb'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mb'].append((running_loss/n_test).numpy())

        
        print('* fit hybrid *')
        res = hybrid_fit(train_data,3)
        data_results['fit_parameters_hybrid'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_hybrid'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = hybrid_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_hybrid'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_hybrid'].append((running_loss/n_test).numpy())
        
        print('* fit nws *')
        res = nws_fit(train_data,1)
        data_results['fit_parameters_nws'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_nws'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = nws_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_nws'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_nws'].append((running_loss/n_test).numpy())
        
        
        print('* fit pwsls *')
        res = pwsls_fit(train_data,2)
        data_results['fit_parameters_pwsls'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_pwsls'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = pwsls_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_pwsls'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_pwsls'].append((running_loss/n_test).numpy())
        
        print('* fit kdh *')
        res = kdh_fit(train_data,3)
        data_results['fit_parameters_kdh'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_kdh'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = kdh_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_kdh'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_kdh'].append((running_loss/n_test).numpy())
        
        print('* fit logistic_regression *')
        X , y = preprocess_logistic_regression(train_data)
        model, intercept, coef = fit_logistic_regression(X,y)
        data_results['fit_parameters_logistic_regression'].append([intercept,coef])

        # Train negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['train_nlp_logistic_regression'].append((nlp/n_train).numpy())
        
        # test data 
        X, y = preprocess_logistic_regression(test_data)
        # Test Accuracy
        accuracy = model.score(X,y)
        data_results['test_acc_logistic_regression'].append(accuracy)
        
        # Test negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['test_nlp_logistic_regression'].append((nlp/n_test).numpy())

         
df = pd.DataFrame(data_results)
df_mean = pd.DataFrame((df.describe().loc['mean']))
df_mean = df_mean.T
test_cols = [col for col in df_mean.columns if 'test' in col]
test_cols = [col for col in test_cols if 'nlp' in col]
d_mf = pd.DataFrame(df_mean[test_cols].describe().loc['mean'])

  0%|                                                     | 0/1 [00:00<?, ?it/s]

*** Fit ***
* loocv block 0 *
* fit mf *
0.3838113681248742
5.446672724037472
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.19it/s]

* loocv block 1 *
* fit mf *
0.3313378751440548
10.0
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


In [4]:
# mb
parameters = []
data_per_agent = []

data_results = {
               'agent': [], 
               'fit_parameters_mf': [], 
               'train_nlp_mf' : [], 
               'test_acc_mf': [],
               'test_nlp_mf': [], 
    
               'fit_parameters_mb': [], 
               'train_nlp_mb' : [], 
               'test_acc_mb': [],
               'test_nlp_mb': [],
    
               'fit_parameters_hybrid': [], 
               'train_nlp_hybrid' : [], 
               'test_acc_hybrid': [],
               'test_nlp_hybrid': [], 
    
               'fit_parameters_nws': [], 
               'train_nlp_nws' : [], 
               'test_acc_nws': [],
               'test_nlp_nws': [], 
    
               'fit_parameters_pwsls': [], 
               'train_nlp_pwsls' : [], 
               'test_acc_pwsls': [],
               'test_nlp_pwsls': [], 
    
               'fit_parameters_kdh': [], 
               'train_nlp_kdh' : [], 
               'test_acc_kdh': [],
               'test_nlp_kdh': [], 
    
               'fit_parameters_logistic_regression': [], 
               'train_nlp_logistic_regression' : [], 
               'test_acc_logistic_regression': [],
               'test_nlp_logistic_regression': [],
 
        }
for agent in tqdm(range(num_of_agents)):
    data = []
    param = configuration_parameters_mb()
    parameters.append(param)

    for i in range(num_of_block):
        # create rewards probs 
        reward_probs = create_reward_probs(num_of_trials,reversal,0.2,0.8)
        df = mb_sim(
                        param,
                        num_of_trials,
                        transtion_probs,
                        reward_probs
        ) 
        data.append(df)
    data_per_agent.append(data)
    print(f'*** Fit ***')
    for n,t in enumerate(cv):
        data_results['agent'].append(agent)
        print(f'* loocv block {n} *')
        train_arr = t[0:-1]
        test_arr = t[-1:]
       
        # split train and test data
        train_data = [data_per_agent[agent][sim] for sim in train_arr]
        train_data = pd.concat(train_data) 
        train_data.reset_index(inplace=True)
        n_train = len(train_data)

        test_data = [data_per_agent[agent][sim] for sim in test_arr]
        test_data = pd.concat(test_data) 
        test_data.reset_index(inplace=True)
        n_test = len(test_data)
        
        print('* fit mf *')
        res = mf_fit(train_data,2)
        data_results['fit_parameters_mf'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mf'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mf_predict(test_data,res.x,)
        
        # Test Accuracy
        data_results['test_acc_mf'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mf'].append((running_loss/n_test).numpy())
        
        print('* fit mb *')
        res = mb_fit(train_data,2)
        data_results['fit_parameters_mb'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mb'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mb_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_mb'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mb'].append((running_loss/n_test).numpy())

        
        print('* fit hybrid *')
        res = hybrid_fit(train_data,3)
        data_results['fit_parameters_hybrid'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_hybrid'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = hybrid_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_hybrid'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_hybrid'].append((running_loss/n_test).numpy())
        
        print('* fit nws *')
        res = nws_fit(train_data,1)
        data_results['fit_parameters_nws'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_nws'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = nws_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_nws'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_nws'].append((running_loss/n_test).numpy())
        
        
        print('* fit pwsls *')
        res = pwsls_fit(train_data,2)
        data_results['fit_parameters_pwsls'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_pwsls'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = pwsls_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_pwsls'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_pwsls'].append((running_loss/n_test).numpy())
        
        print('* fit kdh *')
        res = kdh_fit(train_data,3)
        data_results['fit_parameters_kdh'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_kdh'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = kdh_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_kdh'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_kdh'].append((running_loss/n_test).numpy())
        
        print('* fit logistic_regression *')
        X , y = preprocess_logistic_regression(train_data)
        model, intercept, coef = fit_logistic_regression(X,y)
        data_results['fit_parameters_logistic_regression'].append([intercept,coef])

        # Train negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['train_nlp_logistic_regression'].append((nlp/n_train).numpy())
        
        # test data 
        X, y = preprocess_logistic_regression(test_data)
        # Test Accuracy
        accuracy = model.score(X,y)
        data_results['test_acc_logistic_regression'].append(accuracy)
        
        # Test negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['test_nlp_logistic_regression'].append((nlp/n_test).numpy())

         
df = pd.DataFrame(data_results)
df_mean = pd.DataFrame((df.describe().loc['mean']))
df_mean = df_mean.T
test_cols = [col for col in df_mean.columns if 'test' in col]
test_cols = [col for col in test_cols if 'nlp' in col]
d_mb = pd.DataFrame(df_mean[test_cols].describe().loc['mean'])

  0%|                                                    | 0/50 [00:00<?, ?it/s]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


  2%|▉                                           | 1/50 [00:04<03:45,  4.60s/it]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


  4%|█▊                                          | 2/50 [00:09<04:00,  5.01s/it]

...

 96%|█████████████████████████████████████████▎ | 48/50 [04:00<00:08,  4.48s/it]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


 98%|██████████████████████████████████████████▏| 49/50 [04:06<00:05,  5.01s/it]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


100%|███████████████████████████████████████████| 50/50 [04:12<00:00,  5.05s/it]


In [5]:
# hybrid
parameters = []
data_per_agent = []

data_results = {
               'agent': [], 
               'fit_parameters_mf': [], 
               'train_nlp_mf' : [], 
               'test_acc_mf': [],
               'test_nlp_mf': [], 
    
               'fit_parameters_mb': [], 
               'train_nlp_mb' : [], 
               'test_acc_mb': [],
               'test_nlp_mb': [],
    
               'fit_parameters_hybrid': [], 
               'train_nlp_hybrid' : [], 
               'test_acc_hybrid': [],
               'test_nlp_hybrid': [], 
    
               'fit_parameters_nws': [], 
               'train_nlp_nws' : [], 
               'test_acc_nws': [],
               'test_nlp_nws': [], 
    
               'fit_parameters_pwsls': [], 
               'train_nlp_pwsls' : [], 
               'test_acc_pwsls': [],
               'test_nlp_pwsls': [], 
    
               'fit_parameters_kdh': [], 
               'train_nlp_kdh' : [], 
               'test_acc_kdh': [],
               'test_nlp_kdh': [], 
    
               'fit_parameters_logistic_regression': [], 
               'train_nlp_logistic_regression' : [], 
               'test_acc_logistic_regression': [],
               'test_nlp_logistic_regression': [],
 
        }
for agent in tqdm(range(num_of_agents)):
    data = []
    param = configuration_parameters_hybrid()
    parameters.append(param)

    for i in range(num_of_block):
        # create rewards probs 
        reward_probs = create_reward_probs(num_of_trials,reversal,0.2,0.8)
        df = hybrid_sim(
                        param,
                        num_of_trials,
                        transtion_probs,
                        reward_probs
        ) 
        data.append(df)
    data_per_agent.append(data)
    print(f'*** Fit ***')
    for n,t in enumerate(cv):
        data_results['agent'].append(agent)
        print(f'* loocv block {n} *')
        train_arr = t[0:-1]
        test_arr = t[-1:]
       
        # split train and test data
        train_data = [data_per_agent[agent][sim] for sim in train_arr]
        train_data = pd.concat(train_data) 
        train_data.reset_index(inplace=True)
        n_train = len(train_data)

        test_data = [data_per_agent[agent][sim] for sim in test_arr]
        test_data = pd.concat(test_data) 
        test_data.reset_index(inplace=True)
        n_test = len(test_data)
        
        print('* fit mf *')
        res = mf_fit(train_data,2)
        data_results['fit_parameters_mf'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mf'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mf_predict(test_data,res.x,)
        
        # Test Accuracy
        data_results['test_acc_mf'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mf'].append((running_loss/n_test).numpy())
        
        print('* fit mb *')
        res = mb_fit(train_data,2)
        data_results['fit_parameters_mb'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mb'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mb_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_mb'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mb'].append((running_loss/n_test).numpy())

        
        print('* fit hybrid *')
        res = hybrid_fit(train_data,3)
        data_results['fit_parameters_hybrid'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_hybrid'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = hybrid_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_hybrid'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_hybrid'].append((running_loss/n_test).numpy())
        
        print('* fit nws *')
        res = nws_fit(train_data,1)
        data_results['fit_parameters_nws'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_nws'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = nws_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_nws'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_nws'].append((running_loss/n_test).numpy())
        
        
        print('* fit pwsls *')
        res = pwsls_fit(train_data,2)
        data_results['fit_parameters_pwsls'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_pwsls'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = pwsls_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_pwsls'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_pwsls'].append((running_loss/n_test).numpy())
        
        print('* fit kdh *')
        res = kdh_fit(train_data,3)
        data_results['fit_parameters_kdh'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_kdh'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = kdh_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_kdh'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_kdh'].append((running_loss/n_test).numpy())
        
        print('* fit logistic_regression *')
        X , y = preprocess_logistic_regression(train_data)
        model, intercept, coef = fit_logistic_regression(X,y)
        data_results['fit_parameters_logistic_regression'].append([intercept,coef])

        # Train negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['train_nlp_logistic_regression'].append((nlp/n_train).numpy())
        
        # test data 
        X, y = preprocess_logistic_regression(test_data)
        # Test Accuracy
        accuracy = model.score(X,y)
        data_results['test_acc_logistic_regression'].append(accuracy)
        
        # Test negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['test_nlp_logistic_regression'].append((nlp/n_test).numpy())

         
df = pd.DataFrame(data_results)
df_mean = pd.DataFrame((df.describe().loc['mean']))
df_mean = df_mean.T
test_cols = [col for col in df_mean.columns if 'test' in col]
test_cols = [col for col in test_cols if 'nlp' in col]
d_h = pd.DataFrame(df_mean[test_cols].describe().loc['mean'])

  0%|                                                    | 0/50 [00:00<?, ?it/s]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


  2%|▉                                           | 1/50 [00:08<06:40,  8.17s/it]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


  4%|█▊                                          | 2/50 [00:13<04:58,  6.22s/it]

...

 96%|█████████████████████████████████████████▎ | 48/50 [04:03<00:09,  4.74s/it]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


 98%|██████████████████████████████████████████▏| 49/50 [04:08<00:04,  4.93s/it]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


100%|███████████████████████████████████████████| 50/50 [04:13<00:00,  5.07s/it]


In [6]:
# nws
parameters = []
data_per_agent = []

data_results = {
               'agent': [], 
               'fit_parameters_mf': [], 
               'train_nlp_mf' : [], 
               'test_acc_mf': [],
               'test_nlp_mf': [], 
    
               'fit_parameters_mb': [], 
               'train_nlp_mb' : [], 
               'test_acc_mb': [],
               'test_nlp_mb': [],
    
               'fit_parameters_hybrid': [], 
               'train_nlp_hybrid' : [], 
               'test_acc_hybrid': [],
               'test_nlp_hybrid': [], 
    
               'fit_parameters_nws': [], 
               'train_nlp_nws' : [], 
               'test_acc_nws': [],
               'test_nlp_nws': [], 
    
               'fit_parameters_pwsls': [], 
               'train_nlp_pwsls' : [], 
               'test_acc_pwsls': [],
               'test_nlp_pwsls': [], 
    
               'fit_parameters_kdh': [], 
               'train_nlp_kdh' : [], 
               'test_acc_kdh': [],
               'test_nlp_kdh': [], 
    
               'fit_parameters_logistic_regression': [], 
               'train_nlp_logistic_regression' : [], 
               'test_acc_logistic_regression': [],
               'test_nlp_logistic_regression': [],
 
        }
for agent in tqdm(range(num_of_agents)):
    data = []
    param = configuration_parameters_nws()
    parameters.append(param)

    for i in range(num_of_block):
        # create rewards probs 
        reward_probs = create_reward_probs(num_of_trials,reversal,0.2,0.8)
        df = nws_sim(
                        param,
                        num_of_trials,
                        transtion_probs,
                        reward_probs
        ) 
        data.append(df)
    data_per_agent.append(data)
    print(f'*** Fit ***')
    for n,t in enumerate(cv):
        data_results['agent'].append(agent)
        print(f'* loocv block {n} *')
        train_arr = t[0:-1]
        test_arr = t[-1:]
       
        # split train and test data
        train_data = [data_per_agent[agent][sim] for sim in train_arr]
        train_data = pd.concat(train_data) 
        train_data.reset_index(inplace=True)
        n_train = len(train_data)

        test_data = [data_per_agent[agent][sim] for sim in test_arr]
        test_data = pd.concat(test_data) 
        test_data.reset_index(inplace=True)
        n_test = len(test_data)
        
        print('* fit mf *')
        res = mf_fit(train_data,2)
        data_results['fit_parameters_mf'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mf'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mf_predict(test_data,res.x,)
        
        # Test Accuracy
        data_results['test_acc_mf'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mf'].append((running_loss/n_test).numpy())
        
        print('* fit mb *')
        res = mb_fit(train_data,2)
        data_results['fit_parameters_mb'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mb'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mb_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_mb'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mb'].append((running_loss/n_test).numpy())

        
        print('* fit hybrid *')
        res = hybrid_fit(train_data,3)
        data_results['fit_parameters_hybrid'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_hybrid'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = hybrid_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_hybrid'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_hybrid'].append((running_loss/n_test).numpy())
        
        print('* fit nws *')
        res = nws_fit(train_data,1)
        data_results['fit_parameters_nws'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_nws'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = nws_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_nws'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_nws'].append((running_loss/n_test).numpy())
        
        
        print('* fit pwsls *')
        res = pwsls_fit(train_data,2)
        data_results['fit_parameters_pwsls'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_pwsls'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = pwsls_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_pwsls'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_pwsls'].append((running_loss/n_test).numpy())
        
        print('* fit kdh *')
        res = kdh_fit(train_data,3)
        data_results['fit_parameters_kdh'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_kdh'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = kdh_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_kdh'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_kdh'].append((running_loss/n_test).numpy())
        
        print('* fit logistic_regression *')
        X , y = preprocess_logistic_regression(train_data)
        model, intercept, coef = fit_logistic_regression(X,y)
        data_results['fit_parameters_logistic_regression'].append([intercept,coef])

        # Train negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['train_nlp_logistic_regression'].append((nlp/n_train).numpy())
        
        # test data 
        X, y = preprocess_logistic_regression(test_data)
        # Test Accuracy
        accuracy = model.score(X,y)
        data_results['test_acc_logistic_regression'].append(accuracy)
        
        # Test negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['test_nlp_logistic_regression'].append((nlp/n_test).numpy())

         
df = pd.DataFrame(data_results)
df_mean = pd.DataFrame((df.describe().loc['mean']))
df_mean = df_mean.T
test_cols = [col for col in df_mean.columns if 'test' in col]
test_cols = [col for col in test_cols if 'nlp' in col]
d_nws = pd.DataFrame(df_mean[test_cols].describe().loc['mean'])

  0%|                                                    | 0/50 [00:00<?, ?it/s]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


  2%|▉                                           | 1/50 [00:03<03:08,  3.84s/it]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


  4%|█▊                                          | 2/50 [00:08<03:18,  4.15s/it]

...

 96%|█████████████████████████████████████████▎ | 48/50 [03:25<00:08,  4.15s/it]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


 98%|██████████████████████████████████████████▏| 49/50 [03:28<00:03,  3.98s/it]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


100%|███████████████████████████████████████████| 50/50 [03:33<00:00,  4.27s/it]


In [7]:
# pwsls
parameters = []
data_per_agent = []

data_results = {
               'agent': [], 
               'fit_parameters_mf': [], 
               'train_nlp_mf' : [], 
               'test_acc_mf': [],
               'test_nlp_mf': [], 
    
               'fit_parameters_mb': [], 
               'train_nlp_mb' : [], 
               'test_acc_mb': [],
               'test_nlp_mb': [],
    
               'fit_parameters_hybrid': [], 
               'train_nlp_hybrid' : [], 
               'test_acc_hybrid': [],
               'test_nlp_hybrid': [], 
    
               'fit_parameters_nws': [], 
               'train_nlp_nws' : [], 
               'test_acc_nws': [],
               'test_nlp_nws': [], 
    
               'fit_parameters_pwsls': [], 
               'train_nlp_pwsls' : [], 
               'test_acc_pwsls': [],
               'test_nlp_pwsls': [], 
    
               'fit_parameters_kdh': [], 
               'train_nlp_kdh' : [], 
               'test_acc_kdh': [],
               'test_nlp_kdh': [], 
    
               'fit_parameters_logistic_regression': [], 
               'train_nlp_logistic_regression' : [], 
               'test_acc_logistic_regression': [],
               'test_nlp_logistic_regression': [],
 
        }
for agent in tqdm(range(num_of_agents)):
    data = []
    param = configuration_parameters_pwsls()
    parameters.append(param)

    for i in range(num_of_block):
        # create rewards probs 
        reward_probs = create_reward_probs(num_of_trials,reversal,0.2,0.8)
        df = pwsls_sim(
                        param,
                        num_of_trials,
                        transtion_probs,
                        reward_probs
        ) 
        data.append(df)
    data_per_agent.append(data)
    print(f'*** Fit ***')
    for n,t in enumerate(cv):
        data_results['agent'].append(agent)
        print(f'* loocv block {n} *')
        train_arr = t[0:-1]
        test_arr = t[-1:]
       
        # split train and test data
        train_data = [data_per_agent[agent][sim] for sim in train_arr]
        train_data = pd.concat(train_data) 
        train_data.reset_index(inplace=True)
        n_train = len(train_data)

        test_data = [data_per_agent[agent][sim] for sim in test_arr]
        test_data = pd.concat(test_data) 
        test_data.reset_index(inplace=True)
        n_test = len(test_data)
        
        print('* fit mf *')
        res = mf_fit(train_data,2)
        data_results['fit_parameters_mf'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mf'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mf_predict(test_data,res.x,)
        
        # Test Accuracy
        data_results['test_acc_mf'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mf'].append((running_loss/n_test).numpy())
        
        print('* fit mb *')
        res = mb_fit(train_data,2)
        data_results['fit_parameters_mb'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mb'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mb_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_mb'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mb'].append((running_loss/n_test).numpy())

        
        print('* fit hybrid *')
        res = hybrid_fit(train_data,3)
        data_results['fit_parameters_hybrid'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_hybrid'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = hybrid_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_hybrid'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_hybrid'].append((running_loss/n_test).numpy())
        
        print('* fit nws *')
        res = nws_fit(train_data,1)
        data_results['fit_parameters_nws'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_nws'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = nws_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_nws'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_nws'].append((running_loss/n_test).numpy())
        
        
        print('* fit pwsls *')
        res = pwsls_fit(train_data,2)
        data_results['fit_parameters_pwsls'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_pwsls'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = pwsls_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_pwsls'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_pwsls'].append((running_loss/n_test).numpy())
        
        print('* fit kdh *')
        res = kdh_fit(train_data,3)
        data_results['fit_parameters_kdh'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_kdh'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = kdh_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_kdh'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_kdh'].append((running_loss/n_test).numpy())
        
        print('* fit logistic_regression *')
        X , y = preprocess_logistic_regression(train_data)
        model, intercept, coef = fit_logistic_regression(X,y)
        data_results['fit_parameters_logistic_regression'].append([intercept,coef])

        # Train negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['train_nlp_logistic_regression'].append((nlp/n_train).numpy())
        
        # test data 
        X, y = preprocess_logistic_regression(test_data)
        # Test Accuracy
        accuracy = model.score(X,y)
        data_results['test_acc_logistic_regression'].append(accuracy)
        
        # Test negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['test_nlp_logistic_regression'].append((nlp/n_test).numpy())

         
df = pd.DataFrame(data_results)
df_mean = pd.DataFrame((df.describe().loc['mean']))
df_mean = df_mean.T
test_cols = [col for col in df_mean.columns if 'test' in col]
test_cols = [col for col in test_cols if 'nlp' in col]
d_pwsls = pd.DataFrame(df_mean[test_cols].describe().loc['mean'])

  0%|                                                    | 0/50 [00:00<?, ?it/s]

*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *


  2%|▊                                       | 1/50 [15:11<12:24:11, 911.25s/it]

* fit pwsls *
* fit kdh *
* fit logistic_regression *
*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *


  4%|█▋                                       | 2/50 [15:15<5:02:17, 377.87s/it]

...

* fit logistic_regression *
*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *


 98%|██████████████████████████████████████████▏| 49/50 [33:21<00:03,  3.29s/it]

* fit logistic_regression *
*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *


100%|███████████████████████████████████████████| 50/50 [33:26<00:00, 40.14s/it]

* fit logistic_regression *


In [8]:
# kDH
parameters = []
data_per_agent = []

data_results = {
               'agent': [], 
               'fit_parameters_mf': [], 
               'train_nlp_mf' : [], 
               'test_acc_mf': [],
               'test_nlp_mf': [], 
    
               'fit_parameters_mb': [], 
               'train_nlp_mb' : [], 
               'test_acc_mb': [],
               'test_nlp_mb': [],
    
               'fit_parameters_hybrid': [], 
               'train_nlp_hybrid' : [], 
               'test_acc_hybrid': [],
               'test_nlp_hybrid': [], 
    
               'fit_parameters_nws': [], 
               'train_nlp_nws' : [], 
               'test_acc_nws': [],
               'test_nlp_nws': [], 
    
               'fit_parameters_pwsls': [], 
               'train_nlp_pwsls' : [], 
               'test_acc_pwsls': [],
               'test_nlp_pwsls': [], 
    
               'fit_parameters_kdh': [], 
               'train_nlp_kdh' : [], 
               'test_acc_kdh': [],
               'test_nlp_kdh': [], 
    
               'fit_parameters_logistic_regression': [], 
               'train_nlp_logistic_regression' : [], 
               'test_acc_logistic_regression': [],
               'test_nlp_logistic_regression': [],
 
        }
for agent in tqdm(range(num_of_agents)):
    data = []
    param = configuration_parameters_kdh()
    print(param)
    parameters.append(param)

    for i in range(num_of_block):
        # create rewards probs 
        reward_probs = create_reward_probs(num_of_trials,reversal,0.2,0.8)
        df = kdh_sim(
                        param,
                        num_of_trials,
                        transtion_probs,
                        reward_probs
        ) 
        data.append(df)
    data_per_agent.append(data)
    print(f'*** Fit ***')
    for n,t in enumerate(cv):
        data_results['agent'].append(agent)
        print(f'* loocv block {n} *')
        train_arr = t[0:-1]
        test_arr = t[-1:]
       
        # split train and test data
        train_data = [data_per_agent[agent][sim] for sim in train_arr]
        train_data = pd.concat(train_data) 
        train_data.reset_index(inplace=True)
        n_train = len(train_data)

        test_data = [data_per_agent[agent][sim] for sim in test_arr]
        test_data = pd.concat(test_data) 
        test_data.reset_index(inplace=True)
        n_test = len(test_data)
        
        print('* fit mf *')
        res = mf_fit(train_data,2)
        data_results['fit_parameters_mf'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mf'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mf_predict(test_data,res.x,)
        
        # Test Accuracy
        data_results['test_acc_mf'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mf'].append((running_loss/n_test).numpy())
        
        print('* fit mb *')
        res = mb_fit(train_data,2)
        data_results['fit_parameters_mb'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_mb'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = mb_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_mb'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_mb'].append((running_loss/n_test).numpy())

        
        print('* fit hybrid *')
        res = hybrid_fit(train_data,3)
        data_results['fit_parameters_hybrid'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_hybrid'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = hybrid_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_hybrid'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_hybrid'].append((running_loss/n_test).numpy())
        
        print('* fit nws *')
        res = nws_fit(train_data,1)
        data_results['fit_parameters_nws'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_nws'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = nws_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_nws'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_nws'].append((running_loss/n_test).numpy())
        
        
        print('* fit pwsls *')
        res = pwsls_fit(train_data,2)
        data_results['fit_parameters_pwsls'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_pwsls'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = pwsls_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_pwsls'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_pwsls'].append((running_loss/n_test).numpy())
        
        print('* fit kdh *')
        res = kdh_fit(train_data,3)
        data_results['fit_parameters_kdh'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_kdh'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = kdh_predict(test_data,res.x)
        
        # Test Accuracy
        data_results['test_acc_kdh'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_kdh'].append((running_loss/n_test).numpy())
        
        print('* fit logistic_regression *')
        X , y = preprocess_logistic_regression(train_data)
        model, intercept, coef = fit_logistic_regression(X,y)
        data_results['fit_parameters_logistic_regression'].append([intercept,coef])

        # Train negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['train_nlp_logistic_regression'].append((nlp/n_train).numpy())
        
        # test data 
        X, y = preprocess_logistic_regression(test_data)
        # Test Accuracy
        accuracy = model.score(X,y)
        data_results['test_acc_logistic_regression'].append(accuracy)
        
        # Test negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['test_nlp_logistic_regression'].append((nlp/n_test).numpy())

df = pd.DataFrame(data_results)
df_mean = pd.DataFrame((df.describe().loc['mean']))
df_mean = df_mean.T
test_cols = [col for col in df_mean.columns if 'test' in col]
test_cols = [col for col in test_cols if 'nlp' in col]
d_kDH = pd.DataFrame(df_mean[test_cols].describe().loc['mean'])

  0%|                                                    | 0/50 [00:00<?, ?it/s]

{'p_dh': [0.6516764466053029, 0.22258423263321447, 0.9999236748232119], 'k': 3}
*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


  2%|▉                                           | 1/50 [00:06<05:33,  6.81s/it]

{'p_dh': [0.6424273197382245, 0.6803461911970744, 0.8688799692879601], 'k': 3}
*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


  4%|█▊                                          | 2/50 [00:15<06:31,  8.16s/it]

...

 96%|███████████████████████████████████████▎ | 48/50 [1:06:09<01:00, 30.40s/it]

{'p_dh': [0.8235474831512285, 0.7608254438512208, 0.8389591286015191], 'k': 3}
*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


 98%|████████████████████████████████████████▏| 49/50 [1:06:18<00:23, 23.94s/it]

{'p_dh': [0.03766348093821037, 0.3411456375623345, 0.05448990524534858], 'k': 3}
*** Fit ***
* loocv block 0 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 1 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 2 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 3 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *
* loocv block 4 *
* fit mf *
* fit mb *
* fit hybrid *
* fit nws *
* fit pwsls *
* fit kdh *
* fit logistic_regression *


100%|█████████████████████████████████████████| 50/50 [1:22:20<00:00, 98.80s/it]


In [1]:
d_mf.rename(columns={'mean':'true_mf'},inplace=True) 
d_mb.rename(columns={'mean':'true_mb'},inplace=True) 
d_h.rename(columns={'mean':'true_hybrid'},inplace=True) 
d_nws.rename(columns={'mean':'true_nws'},inplace=True)
d_pwsls.rename(columns={'mean':'true_pwsls'},inplace=True) 
d_kDH.rename(columns={'mean':'true_kdh'},inplace=True)

d = pd.concat([d_mf, d_mb, d_h , d_nws, d_pwsls, d_kDH], axis=1)
d.rename(index={'test_nlp_mf':'fit_mf',
                'test_nlp_mb':'fit_mb',
                'test_nlp_hybrid':'fit_hybrid',
                'test_nlp_nws':'fit_nws',
                'test_nlp_pwsls':'fit_pwsls',
                'test_nlp_kdh':'fit_kdh',
                'test_nlp_logistic_regression':'fit_logistic_regression'},inplace=True)
d.style.background_gradient(cmap ='viridis').set_properties(**{'font-size': '20px'})

NameError: name 'd_mf' is not defined